### **Step 1: Load Data & Split into Train/Test**

In [6]:
import os
import numpy as np
import cv2
from sklearn.model_selection import train_test_split

# ✅ Define Paths (Multiple Folders per Class)
class_0_paths = ["Multiplexer-new/Mode_1/Mode_1-high_bandwidth", "Multiplexer-new/Mode_1/Mode_1-low_bandwidth"]
class_1_paths = ["Multiplexer-new/Mode_2/Mode_2-high_bandwidth", "Multiplexer-new/Mode_2/Mode_2-low_bandwidth"]

# ✅ Function to Load Images from Multiple Folders
def load_images(class_0_folders, class_1_folders, img_size=(256, 256)):
    images, labels = [], []

    # Load Class 0 images from multiple folders
    for folder in class_0_folders:
        if not os.path.exists(folder):  # Check if folder exists
            print(f"Warning: Folder {folder} does not exist!")
            continue
        for img_name in os.listdir(folder):
            img_path = os.path.join(folder, img_name)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            if img is not None:
                img = cv2.resize(img, img_size) / 255.0
                images.append(img)
                labels.append(0)

    # Load Class 1 images from multiple folders
    for folder in class_1_folders:
        if not os.path.exists(folder):  # Check if folder exists
            print(f"Warning: Folder {folder} does not exist!")
            continue
        for img_name in os.listdir(folder):
            img_path = os.path.join(folder, img_name)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            if img is not None:
                img = cv2.resize(img, img_size) / 255.0
                images.append(img)
                labels.append(1)

    # Convert to NumPy arrays
    images = np.array(images).reshape(-1, img_size[0], img_size[1], 1)  # Add channel dimension
    labels = np.array(labels)
    return images, labels

# ✅ Load the dataset from multiple folders
x_data, y_data = load_images(class_0_paths, class_1_paths)

# ✅ Split into Train (80%) and Test (20%)
x_train, x_test, y_train, y_test = train_test_split(
    x_data, y_data, test_size=0.2, stratify=y_data, random_state=42
)

# ✅ Print dataset info
print(f"x_train shape: {x_train.shape}, y_train shape: {y_train.shape}")
print(f"x_test shape: {x_test.shape}, y_test shape: {y_test.shape}")


x_train shape: (335, 256, 256, 1), y_train shape: (335,)
x_test shape: (84, 256, 256, 1), y_test shape: (84,)


### **Step 2: Apply Augmentation on Training Data**

#### Data Augmentation v1.0

In [7]:
# ✅ Stronger Data Augmentation
train_datagen = ImageDataGenerator(
    rotation_range=45,
    width_shift_range=0.4,
    height_shift_range=0.4,
    shear_range=0.4,
    zoom_range=0.5,
    brightness_range=[0.7, 1.3],
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode="nearest"
)

# ✅ Create Augmented Training Dataset
train_ds = train_datagen.flow(x_train, y_train, batch_size=16)

# ✅ Test Data (No Augmentation)
val_ds = (x_test, y_test)


#### Data Augmentation v2.0

In [12]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# ✅ Adjusted Data Augmentation (Less Aggressive)
train_datagen = ImageDataGenerator(
    rotation_range=25,  # 🔻 Reduced from 45 (less distortion)
    width_shift_range=0.2,  # 🔻 Reduced
    height_shift_range=0.2,  # 🔻 Reduced
    shear_range=0.2,  # 🔻 Reduced
    zoom_range=0.3,  # 🔻 Reduced
    brightness_range=[0.8, 1.2],  # 🔻 Less Brightness Change
    horizontal_flip=True,
    vertical_flip=False,  # ❌ Disable vertical flipping for structured images
    fill_mode="nearest"
)

# ✅ Augment Only Training Data
train_ds = train_datagen.flow(x_train, y_train, batch_size=16)

# ✅ Validation Data (No Augmentation)
val_ds = (x_test, y_test)


### **Step 3: Create CNN**

#### CNN v1.0

In [8]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization)

# ✅ Define a Small CNN Model
model = Sequential([
    
    # Convolutional Layer 1
    Conv2D(32, (3,3), activation='relu', input_shape=(256, 256, 1), padding='same'),
    BatchNormalization(),
    MaxPooling2D((2,2)),

    # Convolutional Layer 2
    Conv2D(64, (3,3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D((2,2)),

    # Convolutional Layer 3 (Final Feature Extraction)
    Conv2D(128, (3,3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D((2,2)),

    # Fully Connected Layers
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.4),  # Reduce Overfitting
    Dense(64, activation='relu'),
    Dropout(0.3),
    
    # Output Layer
    Dense(1, activation='sigmoid')  # Binary Classification
])

# ✅ Compile the Model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),  # Lower LR for better convergence
    loss='binary_crossentropy',  # Since we have 2 classes
    metrics=['accuracy']
)

# ✅ Display Model Summary
model.summary()


/Users/nayanthara/Library/Python/3.9/lib/python/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 256, 256, 32)   │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 256, 256, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 128, 128, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 128, 128, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 128, 128, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 64, 64, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 64, 64, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 64, 64, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 32, 32, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 131072)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │    16,777,344 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 16,879,233 (64.39 MB)

 Trainable params: 16,878,785 (64.39 MB)

 Non-trainable params: 448 (1.75 KB)

#### CNN v2.0

In [21]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization

# ✅ Lightweight CNN for Small Dataset
model = Sequential([
    # Convolutional Layer 1
    Conv2D(32, (4,4), activation='relu', input_shape=(256, 256, 1), padding='same'),
    MaxPooling2D((2,2)),

    # Convolutional Layer 2
    Conv2D(32, (4,4), activation='relu', padding='same'),
    MaxPooling2D((2,2)),

    # Flatten & Fully Connected Layers
    Flatten(),
    Dense(64, activation='relu'),
    Dropout(0.4),  # Dropout to prevent overfitting
    Dense(32, activation='relu'),
    Dropout(0.3),

    # Output Layer
    Dense(1, activation='sigmoid')  # Binary Classification
])

# ✅ Compile Model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),  # Start with 1e-3, reduce if needed
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# ✅ Display Model Summary
model.summary()


Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_19 (Conv2D)              │ (None, 256, 256, 32)   │           544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_19 (MaxPooling2D) │ (None, 128, 128, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_20 (Conv2D)              │ (None, 128, 128, 32)   │        16,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_20 (MaxPooling2D) │ (None, 64, 64, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_6 (Flatten)             │ (None, 131072)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_18 (Dense)                │ (None, 64)             │     8,388,672 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_12 (Dropout)            │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_19 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_13 (Dropout)            │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_20 (Dense)                │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 8,407,745 (32.07 MB)

 Trainable params: 8,407,745 (32.07 MB)

 Non-trainable params: 0 (0.00 B)

#### CNN v3.0

In [25]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Dense, Dropout, BatchNormalization, SpatialDropout2D
from keras.regularizers import l2
from keras.callbacks import ReduceLROnPlateau

# ✅ Define the Model
model = Sequential()

# Convolutional Layers
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 1), kernel_regularizer=l2(0.01)))
model.add(BatchNormalization())
model.add(MaxPooling2D(2, 2))
model.add(SpatialDropout2D(0.3))

model.add(Conv2D(64, (3, 3), activation='relu', kernel_regularizer=l2(0.01)))
model.add(BatchNormalization())
model.add(MaxPooling2D(2, 2))

model.add(Conv2D(128, (3, 3), activation='relu', kernel_regularizer=l2(0.01)))
model.add(BatchNormalization())
model.add(MaxPooling2D(2, 2))

# Global Average Pooling (Reduces overfitting)
model.add(GlobalAveragePooling2D())

# Fully Connected Layers
model.add(Dense(128, activation='relu', kernel_regularizer=l2(0.01)))  # 🔻 Reduced neurons (simpler)
model.add(Dropout(0.4))  # 🔻 Prevent overfitting
model.add(Dense(64, activation='relu'))  # 🔻 Reduced layer size
model.add(Dropout(0.3))

# 🔹 Output Layer for **Binary Classification**
model.add(Dense(1, activation='sigmoid'))  # 🔄 Changed from 3 classes → 1 neuron with `sigmoid`

# ✅ Compile the Model for Binary Classification
model.compile(
    optimizer='adam', 
    loss='binary_crossentropy',  # 🔄 Changed from `categorical_crossentropy`
    metrics=['accuracy']
)

# ✅ Learning Rate Scheduler
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6)

# ✅ Display Model Architecture
model.summary()



Model: "sequential_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_24 (Conv2D)              │ (None, 254, 254, 32)   │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_18          │ (None, 254, 254, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_24 (MaxPooling2D) │ (None, 127, 127, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ spatial_dropout2d_1             │ (None, 127, 127, 32)   │             0 │
│ (SpatialDropout2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_25 (Conv2D)              │ (None, 125, 125, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_19          │ (None, 125, 125, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_25 (MaxPooling2D) │ (None, 62, 62, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_26 (Conv2D)              │ (None, 60, 60, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_20          │ (None, 60, 60, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_26 (MaxPooling2D) │ (None, 30, 30, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_1      │ (None, 128)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_23 (Dense)                │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_15 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_24 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_16 (Dropout)            │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_25 (Dense)                │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 118,401 (462.50 KB)

 Trainable params: 117,953 (460.75 KB)

 Non-trainable params: 448 (1.75 KB)

### **Step 3: Train the Model**

In [32]:
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

# ✅ Learning Rate Scheduling & Early Stopping
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6, verbose=1)
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# ✅ Train the model
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10,  # Increase epochs for better learning
    callbacks=[reduce_lr, early_stopping],
    verbose=1
)


Epoch 1/10
21/21 ━━━━━━━━━━━━━━━━━━━━ 10s 446ms/step - accuracy: 0.4904 - loss: 1.0127 - val_accuracy: 0.5357 - val_loss: 0.9779 - learning_rate: 0.0010
Epoch 2/10
21/21 ━━━━━━━━━━━━━━━━━━━━ 9s 427ms/step - accuracy: 0.5229 - loss: 0.9721 - val_accuracy: 0.5357 - val_loss: 0.9502 - learning_rate: 0.0010
Epoch 3/10
21/21 ━━━━━━━━━━━━━━━━━━━━ 9s 424ms/step - accuracy: 0.5659 - loss: 0.9602 - val_accuracy: 0.5357 - val_loss: 0.9261 - learning_rate: 0.0010
Epoch 4/10
21/21 ━━━━━━━━━━━━━━━━━━━━ 9s 413ms/step - accuracy: 0.5090 - loss: 0.9206 - val_accuracy: 0.5357 - val_loss: 0.9050 - learning_rate: 0.0010
Epoch 5/10
21/21 ━━━━━━━━━━━━━━━━━━━━ 9s 413ms/step - accuracy: 0.5680 - loss: 0.8992 - val_accuracy: 0.5357 - val_loss: 0.8866 - learning_rate: 0.0010
Epoch 6/10
21/21 ━━━━━━━━━━━━━━━━━━━━ 10s 462ms/step - accuracy: 0.4931 - loss: 0.8977 - val_accuracy: 0.5357 - val_loss: 0.8677 - learning_rate: 0.0010
Epoch 7/10
21/21 ━━━━━━━━━━━━━━━━━━━━ 10s 452ms/step - accuracy: 0.5094 - loss: 0.8759

### **LIME Code for Multiplexers**

In [35]:
import os
from lime import lime_image
import sys
from skimage.segmentation import mark_boundaries
import matplotlib.pyplot as plt
import cv2
import numpy as np

# ✅ Suppress LIME's Output (to avoid clutter)
class SuppressOutput:
    def __enter__(self):
        self._original_stdout = sys.stdout
        self._original_stderr = sys.stderr
        sys.stdout = open(os.devnull, 'w')
        sys.stderr = open(os.devnull, 'w')

    def __exit__(self, exc_type, exc_value, traceback):
        sys.stdout.close()
        sys.stderr.close()
        sys.stdout = self._original_stdout
        sys.stderr = self._original_stderr

# ✅ Combine Training & Testing Data
all_images = np.concatenate((x_train, x_test), axis=0)
all_labels = np.concatenate((y_train, y_test), axis=0)

# ✅ Define Prediction Function for Binary Classification
def predict_fn(images):
    images = images[:, :, :, 0]  # Extract grayscale channel
    images = np.expand_dims(images, axis=-1)  # Add back single channel
    preds = model.predict(images)  # Get predictions
    
    # 🔹 Convert sigmoid output (0-1) to a two-class probability format
    return np.hstack([1 - preds, preds])  # Converts [0.8] → [0.2, 0.8]

# ✅ Initialize LIME Explainer
explainer = lime_image.LimeImageExplainer()

# ✅ Define Output Directory for LIME Explanations
output_dir = "LIME_Explanations_Multiplexers"
os.makedirs(output_dir, exist_ok=True)

# ✅ Loop Through All Images & Generate LIME Explanations
for i, image in enumerate(all_images):
    # Convert grayscale image to RGB for LIME compatibility
    sample_image = cv2.cvtColor((image.squeeze() * 255).astype('uint8'), cv2.COLOR_GRAY2RGB)

    # ✅ Run LIME Explanation
    with SuppressOutput():  # Suppress LIME's verbose output
        explanation = explainer.explain_instance(
            sample_image.astype('double'),
            predict_fn,
            top_labels=1,  # Only 1 class label needed for binary classification
            hide_color=0,
            num_samples=1000  # Number of perturbations
        )

    # ✅ Get LIME mask & explanation for the predicted class
    predicted_class = explanation.top_labels[0]  # This should be 0 or 1
    temp, mask = explanation.get_image_and_mask(
        predicted_class,  
        positive_only=True,
        num_features=10,
        hide_rest=False
    )

    # ✅ Define Subfolder for Each Class (0 or 1)
    class_folder = os.path.join(output_dir, f"Class_{predicted_class}")
    os.makedirs(class_folder, exist_ok=True)

    # ✅ Save Original Image
    original_image_path = os.path.join(class_folder, f"Multiplexer_Original_Image_{i}.png")
    plt.imsave(original_image_path, sample_image[:, :, 0] / 255.0, cmap='gray')

    # ✅ Save LIME Explanation with Boundaries
    explanation_image_path = os.path.join(class_folder, f"Multiplexer_LIME_Explanation_{i}.png")
    plt.imsave(explanation_image_path, mark_boundaries(temp / 255.0, mask, color=(1, 1, 0)))

    # ✅ Save Mask Directly
    mask_path = os.path.join(class_folder, f"Mask_{i}.png")
    plt.imsave(mask_path, mask, cmap='gray')

print(f"LIME explanations saved in {output_dir}")


LIME explanations saved in LIME_Explanations_Multiplexers
